In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
import os
import warnings

In [3]:
# Suppress warnings
warnings.filterwarnings("ignore")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
# Define label mapping
label_index = {"dry": 0, "normal": 1, "oily": 2, "combination": 3, "sensitive": 4}
index_label = {v: k for k, v in label_index.items()}

# Create dataset dataframe
def create_df(base):
    data = {"images": [], "labels": []}
    for label in os.listdir(base):
        label_path = os.path.join(base, label)
        if label in label_index and os.path.isdir(label_path):
            for img in os.listdir(label_path):
                if img.lower().endswith(('png', 'jpg', 'jpeg')):
                    data["images"].append(os.path.join(label_path, img))
                    data["labels"].append(label_index[label])
    return pd.DataFrame(data)

In [6]:
# Load datasets
train_df = create_df("Data/train")
val_df = create_df("Data/valid")
test_df = create_df("Data/test")

# Split dataset
train_set, temp_set = train_test_split(train_df, test_size=0.2, stratify=train_df['labels'], random_state=42)
val_set, test_set = train_test_split(temp_set, test_size=0.5, stratify=temp_set['labels'], random_state=42)


In [7]:
# Image transformations
IMG_SIZE = 224
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
# Custom dataset class
class SkinDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path, label = self.data.iloc[idx, 0], self.data.iloc[idx, 1]
        img = Image.open(img_path).convert("RGB")
        img = self.transform(img)
        return img, label


In [9]:
# Create dataloaders
BATCH_SIZE = 32
train_ds = SkinDataset(train_set, train_transform)
val_ds = SkinDataset(val_set, transform)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
# Define EfficientNet model
class EfficientNetClassifier(nn.Module):
    def __init__(self, num_classes=5):
        super(EfficientNetClassifier, self).__init__()
        self.model = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
        self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

model = EfficientNetClassifier().to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|█████████████████████████████████████████████████████████████████████████████| 20.5M/20.5M [00:02<00:00, 9.14MB/s]


In [12]:
# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_dl, val_dl, epochs=5):
    for epoch in range(epochs):
        model.train()
        train_loss, correct = 0, 0
        for images, labels in train_dl:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
        train_acc = correct / len(train_dl.dataset)
        print(f"Epoch {epoch+1}, Loss: {train_loss/len(train_dl.dataset):.4f}, Acc: {train_acc:.4f}")

# Train the model
train_model(model, train_dl, val_dl, epochs=5)


Epoch 1, Loss: 0.9854, Acc: 0.5554
Epoch 2, Loss: 0.5908, Acc: 0.7523
Epoch 3, Loss: 0.4039, Acc: 0.8485
Epoch 4, Loss: 0.2977, Acc: 0.8857
Epoch 5, Loss: 0.2341, Acc: 0.9147


<a href="https://colab.research.google.com/github/Randon-Myntra-HackerRamp-21/Acne-type/blob/main/Acne_types_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>